### PENDIENTE
### Cambiar csv por el enlace 
### Comentar cada paso (argumentando lo que hacemos)
### Precio total o precio por metro cuadrado - cual es el mejor para el proyecto?

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, train_test_split


In [8]:
# Cargar el archivo CSV
df = pd.read_csv('https://www.kaggle.com/datasets/thedevastator/prices-characteristics-of-spanish-homes')

# Mostrar las primeras filas para inspeccionar el dataset
df.head()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


In [ ]:
# Mostrar información general del dataset
df.info()

In [ ]:
# Eliminar columnas innecesarias
df.drop(columns=['Unnamed: 0', 'description', 'photo', 'Num Photos', 'recomendado'], inplace=True)

# Quitar símbolos "€ y m²" en 'price' y 'size' y convertir a numérico
df['price'] = pd.to_numeric(df['price'].str.replace('€', '').str.replace(',', ''), errors='coerce')
df['size'] = pd.to_numeric(df['size'].str.replace('m²', '').str.replace(',', ''), errors='coerce')

# Inspeccionar cambios
df[['price', 'size']].head()

In [ ]:
# Eliminar filas con valores nulos
df = df.dropna(subset=['price', 'size', 'rooms', 'bathrooms', 'location', 'region', 'price/m2'])

# Verificar si todavía quedan valores faltantes
print(df.isnull().sum())

In [375]:
# Convertir las columnas 'rooms' y 'bathrooms' a formato numérico
df['rooms'] = pd.to_numeric(df['rooms'], errors='coerce')
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['size'] = pd.to_numeric(df['size'], errors='coerce')

In [ ]:
# Obtener valores únicos en la columna 'type'
unique_types = df['type'].unique()

# Imprimir los valores únicos
print("Valores únicos en 'type':")
print(unique_types)

In [ ]:
# Reemplazar 'piso' por 'pisos' en la columna 'type'
df['type'] = df['type'].replace('piso', 'pisos')

# Verificar los cambios
print(df['type'].value_counts())

In [378]:
# Crear términos de interacción
df['size_rooms_interaction'] = df['size'] * df['rooms']
df['size_bathrooms_interaction'] = df['size'] * df['bathrooms']
df['rooms_bathrooms_interaction'] = df['rooms'] * df['bathrooms']

In [ ]:
# Calcular cuartiles
Q1 = df['size'].quantile(0.25)
Q3 = df['size'].quantile(0.75)
IQR = Q3 - Q1

# Definir límites para detectar valores atípicos
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar los valores atípicos
outliers = df[(df['size'] < lower_bound) | (df['size'] > upper_bound)]

print(f"Número de valores atípicos en 'size': {outliers.shape[0]}")
print(outliers[['size']])

In [ ]:
# Calcular cuartiles para la columna 'rooms'
Q1_rooms = df['rooms'].quantile(0.25)
Q3_rooms = df['rooms'].quantile(0.75)
IQR_rooms = Q3_rooms - Q1_rooms

# Definir límites para detectar valores atípicos
lower_bound_rooms = Q1_rooms - 1.5 * IQR_rooms
upper_bound_rooms = Q3_rooms + 1.5 * IQR_rooms

# Filtrar los valores atípicos
outliers_rooms = df[(df['rooms'] < lower_bound_rooms) | (df['rooms'] > upper_bound_rooms)]

print(f"Número de valores atípicos en 'rooms': {outliers_rooms.shape[0]}")
print(outliers_rooms[['rooms']].head(10))  # Mostrar los primeros 10 valores atípicos

In [ ]:
# Calcular cuartiles para la columna 'bathrooms'
Q1_bathrooms = df['bathrooms'].quantile(0.25)
Q3_bathrooms = df['bathrooms'].quantile(0.75)
IQR_bathrooms = Q3_bathrooms - Q1_bathrooms

# Definir límites para detectar valores atípicos
lower_bound_bathrooms = Q1_bathrooms - 1.5 * IQR_bathrooms
upper_bound_bathrooms = Q3_bathrooms + 1.5 * IQR_bathrooms

# Filtrar los valores atípicos
outliers_bathrooms = df[(df['bathrooms'] < lower_bound_bathrooms) | (df['bathrooms'] > upper_bound_bathrooms)]

print(f"Número de valores atípicos en 'bathrooms': {outliers_bathrooms.shape[0]}")
print(outliers_bathrooms[['bathrooms']].head(10))  # Mostrar los primeros 10 valores atípicos

In [382]:
# Filtrar el DataFrame para eliminar los valores atípicos en 'size', 'rooms', y 'bathrooms'
df_cleaned_final = df[
    (df['size'] >= lower_bound) & (df['size'] <= upper_bound) &
    (df['rooms'] >= lower_bound_rooms) & (df['rooms'] <= upper_bound_rooms) &
    (df['bathrooms'] >= lower_bound_bathrooms) & (df['bathrooms'] <= upper_bound_bathrooms)
]

### Perform Train Test Split

In [ ]:
# Definir características (features) y variable objetivo (target)
X = df_cleaned_final[['size', 'rooms', 'bathrooms', 'type', 'region', 
                       'size_rooms_interaction', 'size_bathrooms_interaction', 'rooms_bathrooms_interaction']]
y = df_cleaned_final['price']  # La variable objetivo

# Preprocesar características categóricas y escalar numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['size', 'rooms', 'bathrooms', 
                                    'size_rooms_interaction', 
                                    'size_bathrooms_interaction', 
                                    'rooms_bathrooms_interaction']),  
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['type', 'region'])  
    ],
    remainder='passthrough'  
)

# Crear un pipeline que primero preprocesa y luego ajusta el modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar el modelo a los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"R² score del modelo en el conjunto de prueba: {r2:.4f}")

### Model Selection / In this exercise we will be using KNN as our predictive model.

In [ ]:
# Definir características (features) y variable objetivo (target)
X = df_cleaned_final[['size', 'rooms', 'bathrooms', 'type', 'region', 
                       'size_rooms_interaction', 'size_bathrooms_interaction', 'rooms_bathrooms_interaction']]
y = df_cleaned_final['price']  # La variable objetivo

# Preprocesar características categóricas y escalar numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['size', 'rooms', 'bathrooms', 
                                    'size_rooms_interaction', 
                                    'size_bathrooms_interaction', 
                                    'rooms_bathrooms_interaction']),  
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['type', 'region'])  
    ],
    remainder='passthrough'  
)

# Crear el modelo KNN
knn_model = KNeighborsRegressor()

# Crear un pipeline que primero preprocesa y luego ajusta el modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', knn_model)
])

# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar el modelo a los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE) con KNN: {mse:.2f}")
print(f"R² score del modelo en el conjunto de prueba: {r2:.4f}")

# Optimización de hiperparámetros con GridSearchCV
param_grid = {
    'regressor__n_neighbors': [3, 5, 7, 10, 15],  # Número de vecinos
    'regressor__weights': ['uniform', 'distance'],  # Peso de los vecinos
}

# Configurar la búsqueda de hiperparámetros
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, verbose=2)

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_knn_model = grid_search.best_estimator_

# Realizar predicciones con el mejor modelo
y_pred_best = best_knn_model.predict(X_test)

# Evaluar el rendimiento del mejor modelo
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print(f"Error cuadrático medio (MSE) con KNN optimizado: {mse_best:.2f}")
print(f"R² score del modelo en el conjunto de prueba: {r2_best:.4f}")